In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

import sqlite3
import re

# Парсинг сайта и настройка базы данных

## 1. Парсинг

In [4]:
# функция очистки текста от лишних символов
def replace_all(text, to_replace):
    for symbol in to_replace:
        text = text.replace(symbol, ' ')
    return text

In [5]:
# ссылки на исполнителей
urls = ['https://txtsong.ru/krovostok/', 'https://txtsong.ru/xaski/', 'https://txtsong.ru/elektroforez/', 'https://txtsong.ru/molchat-doma/', 'https://txtsong.ru/durnoj-vkus/', 'https://txtsong.ru/akvarium/',
       'https://txtsong.ru/skriptonit/', 'https://txtsong.ru/splin/', 'https://txtsong.ru/zemfira/', 'https://txtsong.ru/smoki-mo/', 'https://txtsong.ru/maslo-chernogo-tmina/', 'https://txtsong.ru/2h-company/',
        'https://txtsong.ru/buerak/', 'https://txtsong.ru/agata-kristi/', 'https://txtsong.ru/akvarium/', 'https://txtsong.ru/srub/', 'https://txtsong.ru/aleksandr-f-sklyar/', 'https://txtsong.ru/yanka-dyagileva/',
       'https://txtsong.ru/green-apelsin/', 'https://txtsong.ru/korol-i-shut/', 'https://txtsong.ru/ic3peak/', 'https://txtsong.ru/grechka/']

# создадим список с ссылками на тексты
songs_lists = []

for url in urls:
    songs_list = requests.get(url)
    songs_soup = BeautifulSoup(songs_list.content, 'html.parser')

    for line in songs_soup.find_all('a'):  # вытащим из html адреса с текстами и добавим их в список
        if str(line).find(url[19:]) != -1 and str(line).find('/page/') == -1:
            song_url = line['href']
            songs_lists.append(song_url)

songs_lists = list(set(songs_lists)) # удалим дубли
print(len(songs_lists))

1635


In [6]:
songs_lists[0:5]

['https://txtsong.ru/splin/splin-zvezda-rok-n-rolla/',
 'https://txtsong.ru/skriptonit/skriptonit-tvoj-mikstejp/',
 'https://txtsong.ru/maslo-chernogo-tmina/maslo-chernogo-tmina-nikomu-nevedomyj-mir/',
 'https://txtsong.ru/buerak/buerak-vlyublyonnyj-alfons/',
 'https://txtsong.ru/smoki-mo/smoki-mo-nachalo-igry-chast-2/']

In [7]:
# список с символами для удаления из текстов
symbols_to_del = ['!', '?', '.', ',', ':', '-', '"', '—', '  ', '–', '“', '“', '<p>', '<br/>', '</br>', '<br>', '</p>', '\n', '(', ')', '«', '»', '\u2005', '\u205f', '\xa0', '</em>', '<em>', '\\', 'u200', '<ul>', '<li>', '</li>', '</ul>', "<  Yandex RTB R A 1318229 42  > <div id= yandex_rtb_R A 1318229 42 ></div> <script>window yaContextCb push   =>{  Ya Context AdvManager render {   renderTo 'yandex_rtb_R A 1318229 42'    blockId 'R A 1318229 42'  }  } </script>"]

In [8]:
parsed_songs = {}

for song in songs_lists:
    song_url = requests.get(song)
    song_soup = BeautifulSoup(song_url.content, 'html.parser')
    song_text = str(song_soup.find_all('p')[4]) # находим нужную часть с текстом
    song_text = replace_all(song_text, symbols_to_del)  # очищаем тексты от лишних символов  
    song_text = song_text.rstrip().lstrip() # убираем лишние пробелы в начале и в конце
    parsed_songs[song] = song_text
    time.sleep(1) # создает интервал между действиями, чтобы нас не забанили

In [9]:
len(parsed_songs)

1635

In [11]:
#parsed_songs

## 2. Работа с БД

In [12]:
con = sqlite3.connect('songs.db') # создаем локальную базу данных

In [13]:
cur = con.cursor()
cur.execute('CREATE TABLE song_texts(author, text, url, date)') # создаем таблицу
res = cur.execute('SELECT name FROM sqlite_master')
res.fetchone() # проерим создалась ли таблица

for k, v in parsed_songs.items():   # перебираем ключи и значения и записываем их в таблицу

    sql_insert = """
                    INSERT INTO song_texts (author, text, url, date)
                    VALUES ('{}', '{}', '{}', '{}')
                    """.format(k.split('/')[3], v, k, datetime.today())

    res = cur.execute(sql_insert)
    con.commit()
    
res = cur.execute('SELECT * FROM song_texts')
result = res.fetchall()

In [14]:
result[0]

('splin',
 'Звезда рок н ролла должна умереть Аксиома   Проснись дорогая; прости что не дал досмотреть  Пока ты спала я тихонько смотался из дома узнать что есть жизнь и смерть      Любая строка  это шрам на лице след осколка  Засев в голове он не даст никому постареть  Пока ты спала звезда рок н ролла должна умереть Умереть    Звезда рок н ролла должна умереть без прикола   Три трети пройдя удалиться в четвертую треть   Сегодня так ярко светит луна над балконом что больно смотреть  На эту картину   кухня; пьяный и голый закинувшись всем от чего только можно балдеть  Звезда рок н ролла допивает ром с колой и должна умереть Умереть      Звезда рок н ролла должна умереть очень скоро  Замьютить свой голос расплавиться перегореть  На бешеной коде во время гитарного соло взлететь      Разобьется фонарь  В тишине стадиона будет долго сирена реветь  От укола  Едва ли он сможет когда нибудь снова запеть потому что звезда рок н ролла должна умереть Умереть',
 'https://txtsong.ru/splin/splin-zve

In [16]:
# подготовим тексты 
text_prep_all = []
artist_name = []
for i in range(len(result)):
    text_prep = re.findall('[А-ЯA-Z][^А-ЯA-Z]*', result[i][1]) # делим текст на строки
    text_prep = [a.lower() for a in text_prep] # убираем заглавные буквы
    text_prep = [a.rstrip().rstrip() for a in text_prep] # удаляем лишние пробелы
    text_prep = [a.lstrip() for a in text_prep]
    
    clean_text = [] # убираем короткие и пустые строки
    for val in text_prep:
        if len(val) <= 10:
            pass
        else:
            clean_text.append(val)
    
    if len(clean_text) < 4: # убираем короткие тексты
        pass
    else:
        text_prep_unique = list(set(clean_text)) # оставляем уникальные строки в текстах
        text_prep_str = '\n'.join(text_prep_unique) # добавляем символ табуляции
        text_prep_all.append(text_prep_str) # итоговый список с текстами
        artist_name.append(result[i][0]) # список с именами исполнителей

In [19]:
# создадим датафрейм и сохраним в csv
df_prepped = pd.DataFrame()


df_prepped['text'] = text_prep_all * 3
df_prepped['artist'] = artist_name * 3
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('~/Projects/ML-сервис/lyrics-generator/dif_texts_6.csv', index=False)


print(df_prepped.shape)
df_prepped.head()

(4854, 2)


,artist,text
0,splin,едва ли он сможет когда нибудь снова запеть по...
1,skriptonit,е ёбаные рэперы сука\nследите за новостями\n...
2,maslo-chernogo-tmina,под ногами разрывались петарды\nчаша жамшина\n...
3,buerak,но пароль от карты\nя могу достать тебе любые ...
4,maslo-chernogo-tmina,в тот самый мам не смотри\nи то что оставлю дл...
